In [1]:
import chromadb
import os
import subprocess
from dotenv import load_dotenv
load_dotenv()

os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ['TAVILY_API_KEY'] = os.getenv("TAVILY_API_KEY")
os.environ['GOOGLE_API_KEY'] = os.getenv("GEMINI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

import pandas as pd
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_ollama import OllamaEmbeddings
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import StructuredQueryOutputParser, get_query_constructor_prompt
from langchain.retrievers.self_query.chroma import ChromaTranslator
from langchain_community.utilities import SQLDatabase

embed_model = OllamaEmbeddings(model=os.getenv('OLLAMA_EMBED'))
llama = ChatGroq(model=os.getenv("LLAMA_70B_GROQ"))
gemini = ChatGoogleGenerativeAI(model=os.getenv("GEMINI_FLASH"))

c:\AGENTS\agents_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
db = SQLDatabase.from_uri("sqlite:///skincare_complaints.db")


In [4]:
db.run("SELECT * FROM customer_complaints LIMIT 5")

"[(1, 'Alice Johnson', 'alice.johnson@example.com', 'Anti-Aging Cream', 'Caused skin irritation.', '2024-11-01', 'Open'), (2, 'Bob Smith', 'bob.smith@example.com', 'Sunscreen SPF 50', 'Leaves a white residue.', '2024-11-02', 'In Progress'), (3, 'Charlie Brown', 'charlie.brown@example.com', 'Moisturizing Lotion', 'Bottle was half empty.', '2024-11-03', 'Resolved'), (4, 'Daisy Parker', 'daisy.parker@example.com', 'Acne Treatment Gel', 'No visible results after 3 weeks.', '2024-11-04', 'Open'), (5, 'Evan Taylor', 'evan.taylor@example.com', 'Facial Cleanser', 'Strong chemical smell.', '2024-11-05', 'In Progress')]"

In [5]:
from langchain_community.agent_toolkits import create_sql_agent

In [18]:
agent_executer = create_sql_agent(llama, db=db, 
                                  agent_type='zero-shot-react-description', 
                                  verbose=True
                                  )

In [23]:
res = agent_executer.invoke(
    "check if there is an entry containing email as 'alice.johnson@example.com' and status as open."
    "If entry exists, then return a summary of the entire record containing customer name," 
    "the product, complaint details and status"
    "If there exists no entry then only return Not Found")



> Entering new SQL Agent Executor chain...
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
Action: sql_db_list_tables
Action Input: customer_complaintsNow that I have the list of tables, I can query the schema of the `customer_complaints` table to see what columns it has.

Action: sql_db_schema
Action Input: customer_complaints
CREATE TABLE customer_complaints (
	complaint_id INTEGER, 
	customer_name TEXT NOT NULL, 
	email TEXT NOT NULL, 
	product_name TEXT NOT NULL, 
	complaint_details TEXT NOT NULL, 
	complaint_date TEXT NOT NULL, 
	status TEXT DEFAULT 'Open', 
	PRIMARY KEY (complaint_id)
)

/*
3 rows from customer_complaints table:
complaint_id	customer_name	email	product_name	complaint_details	complaint_date	status
1	Alice Johnson	alice.johnson@example.com	Anti-Aging Cream	Caused skin irritation.	2024-11-01	Open
2	Bob Smith	bob.smith@example.com	Sunscreen SPF 50	Leaves a white residue.	2024

In [24]:
res

{'input': "check if there is an entry containing email as 'alice.johnson@example.com' and status as open.If entry exists, then return a summary of the entire record containing customer name,the product, complaint details and statusIf there exists no entry then only return Not Found",
 'output': 'Alice Johnson, Anti-Aging Cream, Caused skin irritation., Open'}